In [ ]:
import self
import torch
import torch.nn as nn
import torch.optim as optim
import random
import pandas as pd

Ładowanie danych z pliku CSV

In [ ]:
df = pd.read_csv("dane.csv")
X = df[["cecha1", "cecha2", "cecha3"]].values.tolist()
y = df["wartość_docelowa"].values.tolist()
data = list(zip(X, y))

Definicja Modelu JRVS

In [ ]:
class JRVS(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(JRVS, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))


Klasa Trenera Ewolucyjnego

In [ ]:
class EvolutionaryTrainer:
    def __init__(self, model, learning_rate=0.001):
        self.model = model
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    def train(self, data, epochs=10):
        for epoch in range(epochs):
            total_loss = 0
            for x, y in data:
                x, y = torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
                hidden = self.model.init_hidden(x.size(0))

                self.optimizer.zero_grad()
                output, _ = self.model(x, hidden)
                loss = self.criterion(output, y)
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

    def evolve(self):
        mutation_rate = 0.1
        for param in self.model.parameters():
            if random.random() < mutation_rate:
                noise = torch.randn_like(param) * 0.1
                param.data += noise
        print("Model evolved!")

    def save_model(self, path="jrvs_model.pth"):
        torch.save(self.model.state_dict(), path)

    def load_model(self, path="jrvs_model.pth"):
        self.model.load_state_dict(torch.load(path))
        self.model.eval()

Przygotowanie Danych

In [ ]:
data = [
    ([0.1, 0.2, 0.3], [0.5]),
    ([0.2, 0.3, 0.4], [0.6]),
    ([0.3, 0.4, 0.5], [0.7])
]

Inicjalizacja Modelu i Trening

In [ ]:
input_size = 3
hidden_size = 16
output_size = 1

jrvs = JRVS(input_size, hidden_size, output_size)
trainer = EvolutionaryTrainer(jrvs)
trainer.train(data, epochs=10)
trainer.evolve()
trainer.save_model()

Konwersja danych na tensory PyTorch:

In [ ]:
for x_data, y_data in data:
    x = torch.tensor(x_data, dtype=torch.float32).unsqueeze(0)  # Dodaj wymiar batch_size
    y = torch.tensor(y_data, dtype=torch.float32).unsqueeze(0)

Inicjalizacja ukrytego stanu (ważne dla LSTM):

In [ ]:
hidden = self.model.init_hidden(x.size(0))

Propagacja w przód – obliczanie przewidywań modelu:

In [ ]:
output, _ = self.model(x, hidden)

Obliczenie straty:

In [ ]:
loss = self.criterion(output, y)

Propagacja wsteczna (backpropagation) i aktualizacja wag:

In [ ]:
loss.backward()
self.optimizer.step()

Ewolucja modelu

In [ ]:
def evolve(self, mutation_rate=0.1):
    for param in self.model.parameters():
        if random.random() < mutation_rate:
            noise = torch.randn_like(param) * 0.1
            param.data += noise
    print("Model evolved!")

sekcja elitarna

In [ ]:
def evolve_population(population, scores, retain_rate=0.2):
    sorted_models = [x for _, x in sorted(zip(scores, population), reverse=True)]
    num_retain = int(len(sorted_models) * retain_rate)
    return sorted_models[:num_retain]

In [ ]:
trainer.evolve(mutation_rate=0.1)